In [1]:
import numpy as np
import matplotlib.pyplot as plt
from layers import *
from gradientCheck import *
from relativeError import *
%reload_ext autoreload

## 梯度检查： 看affine_backward是否实现正确

In [2]:
np.random.seed(231)

X = np.random.rand(60).reshape(3, 4, 5)

W = np.random.randn(20, 5)

b = np.random.rand(5)

out, cache = affine_forward(X, W, b)

dout = 10 * np.random.randn(*out.shape)

f_X = lambda x: affine_forward(x, W, b)[0]

f_W = lambda w: affine_forward(X, w, b)[0]

f_b = lambda b: affine_forward(X, W, b)[0]

dX_num = numerical_gradient(f_X, X, dout)

dW_num = numerical_gradient(f_W, W, dout)

db_num = numerical_gradient(f_b, b, dout)
print(np.sum(dX_num))

dX, dW, db = affine_backward(dout, cache)

print(np.sum(dX))


print('dX relative error: %f'%(relative_error(dX, dX_num)))

print('dW relative error: %f'%(relative_error(dW, dW_num)))

print('db relative error: %f'%(relative_error(db, db_num)))

162.720377666
162.720377669
dX relative error: 0.000000
dW relative error: 0.000000
db relative error: 0.000000


## 梯度检查： 看relu_backward是否实现正确

In [3]:
X = np.random.randn(3, 4)

print(X)

out, cache = relu_forward(X)

dout = np.random.randn(*out.shape)

dX = relu_backward(dout, cache)

print(dX)

f = lambda x: relu_forward(x)[0]

dX_num = numerical_gradient(f, X, dout)

print(dX_num)

print('dX relative error: %f'%(relative_error(dX, dX_num)))

[[ 0.08706084 -0.39078755 -0.53630667 -0.0694466 ]
 [ 1.19596293 -1.23569345 -0.53806511 -0.6078434 ]
 [ 0.92544992  0.28567297 -0.5000103  -0.88501449]]
[[-0.75379626  0.          0.         -0.        ]
 [-0.93293922  0.          0.          0.        ]
 [-2.07169044 -0.26723653  0.         -0.        ]]
[[-0.75379626  0.          0.          0.        ]
 [-0.93293922  0.          0.          0.        ]
 [-2.07169044 -0.26723653  0.          0.        ]]
dX relative error: 0.000000


## 梯度检查： 看affine_relu_backward是否实现正确

In [4]:
np.random.seed(231)

X = np.random.rand(60).reshape(3, 4, 5)

W = np.random.randn(20, 5)

b = np.random.rand(5)

out, cache = affine_relu_forward(X, W, b)

dout = 10 * np.random.randn(*out.shape)

f_X = lambda x: affine_relu_forward(x, W, b)[0]

f_W = lambda w: affine_relu_forward(X, w, b)[0]

f_b = lambda b: affine_relu_forward(X, W, b)[0]

dX_num = numerical_gradient(f_X, X, dout)

dW_num = numerical_gradient(f_W, W, dout)

db_num = numerical_gradient(f_b, b, dout)
print(np.sum(dX_num))

dX, dW, db = affine_relu_backward(dout, cache)

print(np.sum(dX))


print('dX relative error: %f'%(relative_error(dX, dX_num)))

print('dW relative error: %f'%(relative_error(dW, dW_num)))

print('db relative error: %f'%(relative_error(db, db_num)))

-1.77546717238
-1.77546717233
dX relative error: 0.000000
dW relative error: 0.000000
db relative error: 0.000000


In [13]:
np.random.seed(231)
arr = np.random.randn(5, 4)

In [14]:
arr

array([[ 0.41794341,  1.39710028, -1.78590431, -0.70882773],
       [-0.07472532, -0.77501677, -0.1497979 ,  1.86172902],
       [-1.4255293 , -0.3763567 , -0.34227539,  0.29490764],
       [-0.83732373,  0.95218767,  1.32931659,  0.52465245],
       [-0.14809998,  0.88953195,  0.12444653,  0.99109251]])

In [15]:
index_arr = np.array([3, 1, 0, 2])

In [16]:
arr[:, index_arr]

array([[-0.70882773,  1.39710028,  0.41794341, -1.78590431],
       [ 1.86172902, -0.77501677, -0.07472532, -0.1497979 ],
       [ 0.29490764, -0.3763567 , -1.4255293 , -0.34227539],
       [ 0.52465245,  0.95218767, -0.83732373,  1.32931659],
       [ 0.99109251,  0.88953195, -0.14809998,  0.12444653]])

In [18]:
np.ones((2, 2))

array([[ 1.,  1.],
       [ 1.,  1.]])

In [19]:
np.zeros(2,2)

TypeError: data type not understood

In [20]:
arr

array([[ 0.41794341,  1.39710028, -1.78590431, -0.70882773],
       [-0.07472532, -0.77501677, -0.1497979 ,  1.86172902],
       [-1.4255293 , -0.3763567 , -0.34227539,  0.29490764],
       [-0.83732373,  0.95218767,  1.32931659,  0.52465245],
       [-0.14809998,  0.88953195,  0.12444653,  0.99109251]])

In [21]:
np.ones_like(arr)

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [22]:
arr = np.arange(3).reshape(3, 1)

In [23]:
arr

array([[0],
       [1],
       [2]])

In [24]:
arr2 = np.arange(9).reshape(3, 3)

In [25]:
arr2

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [26]:
arr * arr2

array([[ 0,  0,  0],
       [ 3,  4,  5],
       [12, 14, 16]])

In [27]:
arr

array([[0],
       [1],
       [2]])

In [28]:
arr2

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [29]:
mask = arr > 4

In [30]:
mask

array([[False],
       [False],
       [False]], dtype=bool)